In [9]:
import speech_recognition as sr
from transformers import pipeline
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from textblob import TextBlob
from nltk.probability import FreqDist
import spacy
import nltk

nltk.download('punkt')
nltk.download('stopwords')  # You may also need this for stopwords
# Load spaCy model
nlp = spacy.load('en_core_web_sm')


In [10]:
def capture_speech():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Speak now...")
        audio_data = recognizer.listen(source)
    try:
        text = recognizer.recognize_google(audio_data, language='en-US')
        print("You said: " + text)
        return text
    except sr.UnknownValueError:
        print("Sorry, I could not understand audio")
    except sr.RequestError as e:
        print("Could not request results from Google Speech Recognition service; {0}".format(e))
    return None

def analyze_text(text):
    tokens = word_tokenize(text.lower())
    filtered_tokens = [word for word in tokens if word not in stopwords.words('english')]

    # Keyword Extraction
    fdist = FreqDist(filtered_tokens)
    most_common = fdist.most_common(10)
    keywords = [word for word, count in most_common]

    # Sentiment Analysis
    blob = TextBlob(text)
    sentiment = blob.sentiment
    polarity = sentiment.polarity
    subjectivity = sentiment.subjectivity

    # Named Entity Recognition (NER) using spaCy
    doc = nlp(text)
    entities = [{'text': entity.text, 'label': entity.label_} for entity in doc.ents]

    analysis_results = {
        "filtered_tokens": filtered_tokens,
        "keywords": keywords,
        "sentiment": {"polarity": polarity, "subjectivity": subjectivity},
        "entities": entities
    }

    return analysis_results

def get_user_input():
    user_choice = input("Choose input method (s - speech, t - text, f - file): ")
    if user_choice == 's':
        return capture_speech()
    elif user_choice == 't':
        user_text = input("Enter your text: ")
        return analyze_text(user_text)
    else:
        print("Invalid choice.")
        return None

def generate_response(analysis_results):
    prompt = f"The user expressed {analysis_results['sentiment']['polarity']} sentiment and mentioned {analysis_results['keywords']}. How can I respond in a helpful and informative way?"
    generated_text = pipe(prompt, max_length=100, num_return_sequences=1)[0]['generated_text']
    return generated_text

def main():
    print("Welcome to the AI Communication Coach!")
    text = get_user_input()
    if text:
        print("Processing your input...")
        analysis_results = text
        if isinstance(analysis_results, dict):
            print("Analysis results:")
            print("  - Number of words:", len(analysis_results["filtered_tokens"]))
            print("  - Keywords:", ", ".join(analysis_results["keywords"]))
            sentiment = analysis_results["sentiment"]
            if sentiment["polarity"] > 0:
                print("  - Sentiment: Positive")
            elif sentiment["polarity"] < 0:
                print("  - Sentiment: Negative")
            else:
                print("  - Sentiment: Neutral")


In [ ]:
main()